## GeoEnrichment

This is a key capability you can use in your data science projects. It helps you to get facts about a specific area. The area can be defined by administrative boundaries, or be a result of gemoetric operations such as distance buffer or drive time from a location. Available facts are stored in multiple datasets and reflect population, income, housing, consumer behavior, and the natural environment. Much of the remainder of this notebook is based on https://developers.arcgis.com/python/guide/part1-introduction-to-geoenrichment/ and subsequent documentation sections.

The main method is **enrich()**: it retrieves info for the specified area.
The arcgis.geoenrichment module can help you create geometries to which enrich() can be later applied.

**Examples:** 
    - A wildfire analyst generates a map of the dynamics and extent of forect fires: you need to quickly determine who lives there and what their mobility charatceristics are. 
    - A company is looking for a location of a new store: you need to determine who lives in the vicinity and what they typically buy.

In [1]:
#necessary for python 3.10 since Iterable in collections was depricated in 3.9
#source https://stackoverflow.com/questions/72032032/importerror-cannot-import-name-iterable-from-collections-in-python
# import collections.abc
# #hyper needs the four following aliases to be done manually.
# collections.Iterable = collections.abc.Iterable
# collections.Mapping = collections.abc.Mapping
# collections.MutableSet = collections.abc.MutableSet
# collections.MutableMapping = collections.abc.MutableMapping
# #Now import hyper
# import hyper

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import *
import pandas as pd

#gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
print("Enter the Client ID/APP ID generated from Arc GIS:")
app_id = input()
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)
arcgis.__version__
# Elm5V3upnnV17Q3r

Enter the Client ID/APP ID generated from Arc GIS:


 Elm5V3upnnV17Q3r


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=nYNo6lAEaD8cXzyii6Df1r43EMlPlt&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2.1.0'

### What information is available for your country of interest?

In [2]:
# What is available for the US?

usa = Country.get('US')
usa.properties.datasets

['USA_ESRI_2022',
 'USA_ACS_2022',
 'USA_ASR_2022',
 'USA_CRM_2022',
 'USA_DATAAXLE_2022',
 'USA_SAFEGRAPH_2022',
 'USA_TRFCNT_2022',
 'USA_ESRI_2021',
 'USA_ACS_2021',
 'USA_ASR_2021',
 'USA_CRM_2021',
 'USA_RMP_2021',
 'Landscape']

In [3]:
type(usa)

arcgis.geoenrichment.enrichment.Country

### Geoenrichment integrates data from many databases:

* ACS: American Community Survey (https://www.census.gov/programs-surveys/acs, https://doc.arcgis.com/en/esri-demographics/data/acs.htm, http://suave2.sdsc.edu/gallery/sdhhsa)
* ASR: Age, Sex, Race (https://www.census.gov/newsroom/press-kits/2020/population-estimates-detailed.html)
* CRM: Crimes (https://doc.arcgis.com/en/esri-demographics/data/crime-indexes.htm)
* RMP: Retail MarketPlace (https://downloads.esri.com/esri_content_doc/dbl/us/Var_List_Retail-MarketPlace_Summer2020.pdf, https://doc.arcgis.com/en/esri-demographics/data/market-potential.htm)
* Safegraph: 5 million point locations for any transactions (https://www.esri.com/arcgis-blog/products/bus-analyst/data-management/why-and-when-to-use-safegraph-data-in-your-analysis/, https://doc.arcgis.com/en/esri-demographics/data/business.htm)
* Traffic Counts (https://doc.arcgis.com/en/esri-demographics/data/traffic-counts.htm)

Global coverage of geoenrichment: https://doc.arcgis.com/en/arcgis-online/reference/geoenrichment-coverage.htm

Standard geography levels: https://geoenrichdev.arcgis.com/arcgis/rest/services/World/GeoenrichmentServer/Geoenrichment/StandardGeographyLevels

You can also also find variables for geoenrichment with the ESRI Demographics Data Browser at https://doc.arcgis.com/en/esri-demographics/data/data-browser.htm (may be easier!)

In [4]:
# Listing  specific data collections
df = usa.data_collections
df

,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
1yearincrements,1yearincrements.AGE0_CY,2022 Population Age <1,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE1_CY,2022 Population Age 1,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE2_CY,2022 Population Age 2,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE3_CY,2022 Population Age 3,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE4_CY,2022 Population Age 4,2022 Age: 1 Year Increments (Esri),2022
...,...,...,...,...
yearmovedin,yearmovedin.MOEMEDYRMV,2020 Median Year Householder Moved In MOE (ACS...,2016-2020 Year Householder Moved In (ACS),2016-2020
yearmovedin,yearmovedin.RELMEDYRMV,2020 Median Year Householder Moved In REL (ACS...,2016-2020 Year Householder Moved In (ACS),2016-2020
yearmovedin,yearmovedin.ACSOWNER,2020 Owner Households (ACS 5-Yr),2016-2020 Key Demographic Indicators (ACS),2016-2020


In [5]:
df.reset_index(inplace=True)

In [7]:
#df["vars"]= df["analysisVariable"].str.split(".", n = 1, expand = False)
#df['Length'] = df['vars'].str.len()
# new data frame with split value columns

In [6]:
# Splitting the analysisVariable to match with Variable
variables = df["analysisVariable"].str.split(".", n = 1, expand = True)
df["Variable"]= variables[1]
df

,dataCollectionID,analysisVariable,alias,fieldCategory,vintage,Variable
0,1yearincrements,1yearincrements.AGE0_CY,2022 Population Age <1,2022 Age: 1 Year Increments (Esri),2022,AGE0_CY
1,1yearincrements,1yearincrements.AGE1_CY,2022 Population Age 1,2022 Age: 1 Year Increments (Esri),2022,AGE1_CY
2,1yearincrements,1yearincrements.AGE2_CY,2022 Population Age 2,2022 Age: 1 Year Increments (Esri),2022,AGE2_CY
3,1yearincrements,1yearincrements.AGE3_CY,2022 Population Age 3,2022 Age: 1 Year Increments (Esri),2022,AGE3_CY
4,1yearincrements,1yearincrements.AGE4_CY,2022 Population Age 4,2022 Age: 1 Year Increments (Esri),2022,AGE4_CY
...,...,...,...,...,...,...
18941,yearmovedin,yearmovedin.MOEMEDYRMV,2020 Median Year Householder Moved In MOE (ACS...,2016-2020 Year Householder Moved In (ACS),2016-2020,MOEMEDYRMV
18942,yearmovedin,yearmovedin.RELMEDYRMV,2020 Median Year Householder Moved In REL (ACS...,2016-2020 Year Householder Moved In (ACS),2016-2020,RELMEDYRMV
18943,yearmovedin,yearmovedin.ACSOWNER,2020 Owner Households (ACS 5-Yr),2016-2020 Key Demographic Indicators (ACS),2016-2020,ACSOWNER
18944,yearmovedin,yearmovedin.MOEOWNER,2020 Owner Households MOE (ACS 5-Yr),2016-2020 Key Demographic Indicators (ACS),2016-2020,MOEOWNER


In [7]:
# Read excel file with sheet name
dict_df = pd.read_excel('Variables to Use in Food Swamp and Opportunities Maps 2-2-23.xlsx', 
                   sheet_name=['Esri Consumer Spending Data ','Esri Business Data','Esri Market Potential Data','Esri Demographics'])

In [8]:
def process_the_collection(segment):
    print(f"==== Results for Segment: {segment} ========")
    vars_df = dict_df.get(segment)
    #print(f"Lenth of the variable df: {len(vars_df)}")
    ## Variables to use column has extra space, we will have to strip that out
    #vars_df['Variables to use'] = vars_df['Variables to use'].str.strip()
    vars_df = vars_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # drop Nan from the dataframe
    vars_df = vars_df.dropna(subset=['Variables to use'])
    print(f"Number of unique variables: {vars_df['Variables to use'].unique().size}")
    # We need dataCollectionID to get all the relevant code information.
    # Lets join the vars given by domain expert with existing USA collections.
    #join_df = df.join(consumer_spending_vars_df.set_index('Variables to use'), on='Variable')
    #len(join_df)
    merge_df = pd.merge(df, vars_df, left_on='Variable', right_on='Variables to use')
    print(f"Length of the joined DF: {len(merge_df)}")
    print(f"For this Segment, all the dataCollections are:\n {list(merge_df['dataCollectionID'].unique())}")
    print("\n")

In [9]:
# consumer_spending_vars_df = dict_df.get('Esri Consumer Spending Data ')
# business_data_vars_df = dict_df.get('Esri Business Data')
# market_potential_vars_df = dict_df.get('Esri Market Potential Data')
# demographics_vars_df = dict_df.get('Esri Demographics')

In [10]:
process_the_collection('Esri Consumer Spending Data ')
process_the_collection('Esri Business Data')
process_the_collection('Esri Market Potential Data')
process_the_collection('Esri Demographics')

==== Results for Segment: Esri Consumer Spending Data  ========
Number of unique variables: 360
Length of the joined DF: 360
For this Segment, all the dataCollections are:
 ['entertainment', 'food', 'HousingHousehold', 'SpendingTotal', 'TravelCEX']


==== Results for Segment: Esri Business Data ========
Number of unique variables: 99
Length of the joined DF: 201
For this Segment, all the dataCollections are:
 ['businesses', 'industrybysiccode', 'Health', 'industrybynaicscode', 'employees', 'sales']


==== Results for Segment: Esri Market Potential Data ========
Number of unique variables: 1264
Length of the joined DF: 1196
For this Segment, all the dataCollections are:
 ['AtRisk', 'PetsPetProducts', 'BabyProductsToysGames', 'ElectronicsInternet', 'GroceryAlcoholicBeverages', 'HealthPersonalCare', 'HomeImprovementGardenLawn', 'HouseholdGoodsFurnitureAppliances', 'LeisureActivitiesLifestyle', 'MediaMagazinesNewspapers', 'MediaRadioOtherAudio', 'MediaTVViewing', 'PsychographicsAdvertising

In [11]:
#!pip install hyper

In [12]:
#!pip show hyper

In [13]:
#df.dropna(subset=['fieldCategory'],inplace=True)

In [14]:

#df[df["analysisVariable"].str.contains("MP03026h_B")]
#df[df["dataCollectionID"].str.contains("BabyProductsToysGames")]

In [15]:

#df[df["fieldCategory"].str.contains("Market Potential")]["dataCollectionID"].unique()

In [16]:
# df = usa.data_collections
# df = df.dropna(subset=['fieldCategory'])
# df = df[df["fieldCategory"].str.contains("Business")]
# #df["fieldCategory"].unique()
# df

In [17]:

#df[df['dataCollectionID']=='businesses']
#Age,businesses

In [18]:
usa.levels

,level_name,singular_name,plural_name,alias,level_id,admin_level
0,block_groups,Block Group,Block Groups,Block Groups,US.BlockGroups,
1,tracts,Census Tract,Census Tracts,Census Tracts,US.Tracts,
2,places,Place,Places,Cities and Towns (Places),US.Places,
3,zip5,ZIP Code,ZIP Codes,ZIP Codes,US.ZIP5,Admin4
4,csd,County Subdivision,County Subdivisions,County Subdivisions,US.CSD,
5,counties,County,Counties,Counties,US.Counties,Admin3
6,cbsa,CBSA,CBSAs,CBSAs,US.CBSA,
7,cd,Congressional District,Congressional Districts,Congressional Districts,US.CD,
8,dma,DMA,DMAs,DMAs,US.DMA,
9,states,State,States,States,US.States,Admin2


In [19]:
usa.enrich_variables

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
0,AGE0_CY,2022 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY,2022 Total Population Age <1 (Esri),2022,count
1,AGE1_CY,2022 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY,2022 Total Population Age 1 (Esri),2022,count
2,AGE2_CY,2022 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY,2022 Total Population Age 2 (Esri),2022,count
3,AGE3_CY,2022 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY,2022 Total Population Age 3 (Esri),2022,count
4,AGE4_CY,2022 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY,2022 Total Population Age 4 (Esri),2022,count
...,...,...,...,...,...,...,...,...
18941,MOEMEDYRMV,2020 Median Year Householder Moved In MOE (ACS...,yearmovedin,yearmovedin.MOEMEDYRMV,yearmovedin_MOEMEDYRMV,2020 Median Year Householder Moved into Unit M...,2016-2020,count
18942,RELMEDYRMV,2020 Median Year Householder Moved In REL (ACS...,yearmovedin,yearmovedin.RELMEDYRMV,yearmovedin_RELMEDYRMV,2020 Median Year Householder Moved into Unit R...,2016-2020,count
18943,ACSOWNER,2020 Owner Households (ACS 5-Yr),yearmovedin,yearmovedin.ACSOWNER,yearmovedin_ACSOWNER,2020 Owner Households (ACS 5-Yr),2016-2020,count
18944,MOEOWNER,2020 Owner Households MOE (ACS 5-Yr),yearmovedin,yearmovedin.MOEOWNER,yearmovedin_MOEOWNER,2020 Owner Households MOE (ACS 5-Yr),2016-2020,count


In [20]:
usa.travel_modes

,name,alias,description,type,impedance,impedance_category,time_attribute_name,distance_attribute_name,travel_mode_id,travel_mode_dict
0,driving_time,Driving Time,Models the movement of cars and other similar ...,AUTOMOBILE,TravelTime,temporal,TravelTime,Kilometers,FEgifRtFndKNcJMJ,"{""attributeParameterValues"": [{""attributeName""..."
1,driving_distance,Driving Distance,Models the movement of cars and other similar ...,AUTOMOBILE,Kilometers,distance,TravelTime,Kilometers,iKjmHuBSIqdEfOVr,"{""attributeParameterValues"": [{""attributeName""..."
2,trucking_time,Trucking Time,Models basic truck travel by preferring design...,TRUCK,TruckTravelTime,temporal,TruckTravelTime,Kilometers,ZzzRtYcPLjXFBKwr,"{""attributeParameterValues"": [{""attributeName""..."
3,trucking_distance,Trucking Distance,Models basic truck travel by preferring design...,TRUCK,Kilometers,distance,TruckTravelTime,Kilometers,UBaNfFWeKcrRVYIo,"{""attributeParameterValues"": [{""attributeName""..."
4,walking_time,Walking Time,Follows paths and roads that allow pedestrian ...,WALK,WalkTime,temporal,WalkTime,Kilometers,caFAgoThrvUpkFBW,"{""attributeParameterValues"": [{""attributeName""..."
5,walking_distance,Walking Distance,Follows paths and roads that allow pedestrian ...,WALK,Kilometers,distance,WalkTime,Kilometers,yFuMFwIYblqKEefX,"{""attributeParameterValues"": [{""attributeName""..."
6,rural_driving_time,Rural Driving Time,Models the movement of cars and other similar ...,AUTOMOBILE,TravelTime,temporal,TravelTime,Kilometers,NmNhNDUwZmE1YTlj,"{""attributeParameterValues"": [{""attributeName""..."
7,rural_driving_distance,Rural Driving Distance,Models the movement of cars and other similar ...,AUTOMOBILE,Kilometers,distance,TravelTime,Kilometers,Yzk3NjI1NTU5NjVj,"{""attributeParameterValues"": [{""attributeName""..."


In [21]:
# Discover named geographies and level of detail

#sandiego_in_usa = usa.search('San Diego')
#print("number of San Diego's in the US: " + str(len(sandiego_in_usa)))

# list a few of them
#sandiego_in_usa[:30]

In [22]:
#type(sandiego_in_usa[0])



In [23]:
usa.reports

,id,title,categories,formats
0,census2010_profile,2010 Census Profile,[Demographics],"[pdf, xlsx]"
1,acs_housing,ACS Housing Summary,[Demographics],"[pdf, xlsx]"
2,acs_keyfacts,ACS Key Population & Household Facts,[Demographics],"[pdf, xlsx]"
3,acs_population,ACS Population Summary,[Demographics],"[pdf, xlsx]"
4,55plus,Age 50+ Profile,[Demographics],"[pdf, xlsx]"
5,agesexrace,Age by Sex by Race Profile,[Demographics],"[pdf, xlsx]"
6,agesex,Age by Sex Profile,[Demographics],"[pdf, xlsx]"
7,cex_auto,Automotive Aftermarket Expenditures,[Consumer Spending],"[pdf, xlsx]"
8,business_loc,Business Locator,[Business],"[pdf, xlsx]"
9,business_summary,Business Summary,[Business],"[pdf, xlsx]"


In [24]:
#usa.subgeographies.states.values()

In [25]:
list(usa.subgeographies.cbsa.values())[:10]

Exception: User does not have permissions to access this resource.
(Error Code: 403)

In [ ]:
usa.dataset

In [ ]:
usa.hierarchies

In [ ]:
# let's put them on a map
#usamap = gis.map('United States', zoomlevel=4)
#usamap

In [ ]:
imperial_in_usa = usa.search("Imperial")

In [ ]:
imperial_in_usa[0]

In [ ]:
sd_county_block_groups = list(usa.subgeographies.states['California'].counties['San_Diego_County'].block_groups)
print("Number of block_groups in San_Diego_County",len(sd_county_block_groups))
print("Samples:")
sd_county_block_groups[:10]

In [ ]:
#usa.subgeographies.states['California'].counties['San_Diego_County'].block_groups

In [ ]:
sd_county_tracts = list(usa.subgeographies.states['California'].counties['San_Diego_County'].tracts)
print("Number of tracts in San_Diego_County",len(sd_county_tracts))
print("Samples:")
sd_county_tracts[:10]

In [ ]:
california_counties = list(usa.subgeographies.states['California'].counties)
california_counties[:10]

In [ ]:
list(usa.subgeographies.states['California'].counties['Imperial_County'].tracts)[0:10]

In [ ]:
usa.subgeographies.states['California'].counties['Imperial_County'].tracts

In [ ]:
list(usa.subgeographies.states['California'].places)[:10]

In [ ]:
usa.subgeographies.states['California'].dma['San_Diego,_CA']

In [ ]:
usa.subgeographies.states['California'].csd['San_Diego_CCD']

In [ ]:
list(usa.subgeographies.states['California'].counties["San_Diego_County"].zip5)[:10]

In [ ]:
usa.subgeographies.states['California'].counties["San_Diego_County"].block_groups['060730009.011']

In [ ]:
def stats_for_geaographic_area()

In [ ]:
sd_tracts = usa.subgeographies.states['California'].counties['San_Diego_County'].tracts
sd_list=list(sd_tracts.values())


In [ ]:
sd_tracts_enriched = enrich(study_areas=sd_tracts, data_collections=['food'])
sd_tracts_enriched

In [3]:
sd_block_groups = usa.subgeographies.states['California'].counties['San_Diego_County']
sd_block_groups_list = list(sd_block_groups.values())

In [4]:
sd_block_groups_list[:10]

[<NamedArea name:"United States" area_id="060730009011", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730009012", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730009013", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730009021", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730009022", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730009023", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730010001", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730010002", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area_id="060730010003", level="US.BlockGroups", country="United States">,
 <NamedArea name:"United States" area

In [5]:
sd_block_groups_enriched = enrich(study_areas=sd_block_groups, data_collections=['food'])
sd_block_groups_enriched

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,x2001_x,x2001_a,...,x1130fy_x,x1130fy_a,x1130fy_i,x1002fy_x,x1002fy_a,x1002fy_i,x1003fy_x,x1003fy_a,x1003fy_i,SHAPE
0,US.BlockGroups,060730009.011,060730009011,USA,Query:US.BlockGroups,2.191,2.576,1,558364.0,724.21,...,4484900.0,5932.41,118.0,10636516.0,14069.47,115.0,6151616.0,8137.06,113.0,"{""rings"": [[[-117.13760189952215, 32.755287766..."
1,US.BlockGroups,060730009.012,060730009012,USA,Query:US.BlockGroups,2.191,2.576,1,283254.0,560.9,...,2146540.0,4362.89,87.0,5160207.0,10488.23,86.0,3013668.0,6125.34,85.0,"{""rings"": [[[-117.13758289925558, 32.752193766..."
2,US.BlockGroups,060730009.013,060730009013,USA,Query:US.BlockGroups,2.191,2.576,1,310418.0,629.65,...,2432512.0,4816.86,96.0,5608004.0,11104.96,91.0,3175492.0,6288.1,87.0,"{""rings"": [[[-117.1400498994012, 32.7484517657..."
3,US.BlockGroups,060730009.021,060730009021,USA,Query:US.BlockGroups,2.191,2.576,1,231341.0,363.17,...,1758765.0,2823.06,56.0,4372303.0,7018.14,57.0,2613538.0,4195.08,58.0,"{""rings"": [[[-117.14252390011785, 32.750313766..."
4,US.BlockGroups,060730009.022,060730009022,USA,Query:US.BlockGroups,2.191,2.576,1,323288.0,718.42,...,2422661.0,5100.34,102.0,5585293.0,11758.51,96.0,3162632.0,6658.17,92.0,"{""rings"": [[[-117.14375390035434, 32.750309765..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,US.BlockGroups,060730117.003,060730117003,USA,Query:US.BlockGroups,2.191,2.576,1,124477.0,181.19,...,1012410.0,1486.65,30.0,2516860.0,3695.83,30.0,1504449.0,2209.18,31.0,"{""rings"": [[[-117.10370888785712, 32.669078758..."
996,US.BlockGroups,060730117.004,060730117004,USA,Query:US.BlockGroups,2.191,2.576,1,256318.0,461.0,...,2286137.0,3887.99,78.0,5495795.0,9346.59,77.0,3209658.0,5458.6,76.0,"{""rings"": [[[-117.10454288825848, 32.672659758..."
997,US.BlockGroups,060730127.001,060730127001,USA,Query:US.BlockGroups,2.191,2.576,1,387924.0,500.55,...,3062261.0,3842.23,77.0,7315781.0,9179.15,75.0,4253520.0,5336.91,74.0,"{""rings"": [[[-117.07722188114349, 32.634375755..."
998,US.BlockGroups,060730127.002,060730127002,USA,Query:US.BlockGroups,2.191,2.576,1,279663.0,405.31,...,2357617.0,3492.77,70.0,5703951.0,8450.3,69.0,3346334.0,4957.53,69.0,"{""rings"": [[[-117.08672788222613, 32.635556754..."


In [6]:
enrich(study_areas=[usa.subgeographies.states['California'].zip5['92122']],analysis_variables=['X1135_X'])

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,x1135_x,SHAPE
0,US.ZIP5,San Diego,92122,USA,Query:US.ZIP5,2.191,2.576,1,2032199.0,"{""rings"": [[[-117.23309999984802, 32.837879969..."


In [7]:
type(sd_block_groups_enriched)

pandas.core.frame.DataFrame

In [10]:
sd_block_groups_enriched.shape[0]

1000